In [1]:
import datetime
import fetcher
import pandas as pd
from finta import TA

startDate = datetime.date(2023, 12, 1)
endDate = datetime.date(2024, 5, 12)
#endDate = datetime.date.today()
Stocks_list = ["POWL"]


focus_list_vol = {}
focus_list_ADR = {}
focus_list_3Month = {}
focus_list_1Month = {}
focus_list_RS = {}
focus_list_RS_min_max = {}
focus_list_EMA10 = []
focus_candleSize = {}

index_df = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

for ticker in Stocks_list:

    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    #Neglecting tickers with len < 66
    if len(df) < 66:
        continue
    
    # Avg Volume*Price
    df_temp = df.iloc[-50:]
    avg_dollar_vol = 0
    for index, row in df_temp.iterrows():
        avg_dollar_vol += (
            df_temp.loc[index, "Volume"] * df_temp.loc[index, "Close"]
        ) / 50
    if avg_dollar_vol >= 20000000 and avg_dollar_vol < 100000000:
        focus_list_vol[ticker] = avg_dollar_vol
 
    # ADR%
    df_temp = df.iloc[-20:]
    sum = 0
    for index, row in df_temp.iterrows():
        sum += (df_temp.loc[index, "High"] / df_temp.loc[index, "Low"]) / 20

    ADR = 100 * (sum - 1)
    if ADR >= 3.5:
        focus_list_ADR[ticker] = ADR

    # Low near to or less than EMA10
    if df.tail(1)["Low"].iloc[-1] <= (1+ADR/300)*(TA.EMA(df, period = 10).tail(1).iloc[-1]):
        focus_list_EMA10.append(ticker)

    focus_list_RS_min_max[ticker] = 0
    # Relative Strength low-high ratio 3M
    index_df_temp = index_df.iloc[-66:]
    df_temp = df.iloc[-66:]
    for i in range(1,4):
        index_df_temp_1M = index_df_temp[(i-1)*22:i*22]
        df_temp_1M = df_temp[(i-1)*22:i*22]
        RS = df_temp_1M["Close"].values/index_df_temp_1M["Close"].values
        min = RS[0]
        max = RS[1]
        for i in range(len(RS)-1):
            for j in range(i+1, len(RS)):
                if RS[j] > RS[i]:
                    if (RS[j]-RS[i])/RS[i] * 100 > (max-min)/min *100:
                        max = RS[j]
                        min = RS[i]

        RS_change_perc = (max-min)/min * 100
        if i ==3:
            focus_list_RS_min_max[ticker] += .5*RS_change_perc
        else:
            focus_list_RS_min_max[ticker] += .25*RS_change_perc

    #Check candle size less than ADR%
    change = (df["High"][-1]/df["Low"][-1] -1)*100
    if change <= ADR*1.5 and change <= 7:
        focus_candleSize[ticker] = change

    
    # Relative Strength 1M
    """index_df_temp = index_df.iloc[-22:]
    df_temp = df.iloc[-22:]

    start_RS = df_temp.iloc[0]["Close"]/index_df_temp.iloc[0]["Close"]
    end_RS = df_temp.iloc[-1]["Close"]/index_df_temp.iloc[-1]["Close"]

    RS_change_perc = (end_RS-start_RS)/start_RS *100

    focus_list_RS[ticker] = RS_change_perc"""


    # 1Month% performance
    """df_temp = df.iloc[-22:]
    df_low = df_temp["Low"]
    df_high = df_temp["High"]
    low = df_low[0]
    high = df_high[1]
    max_change_perc = 0
    for i in range(len(df_low)-1):
        for j in range(i, len(df_high)):
            change_perc = (df_high[j]-df_low[i])/df_low[i] * 100
            if change_perc > max_change_perc:
                max_change_perc = change_perc
        
    focus_list_1Month[ticker] = change_perc"""
    """if change_perc >= 20 :
        focus_list_1Month[ticker] = change_perc"""

    # 3Month% performance
    
    """df_temp = df.iloc[-66:]
    df_low = df_temp["Low"]
    df_high = df_temp["High"]
    low = df_low[0]
    high = df_high[1]
    max_change_perc = 0
    for i in range(len(df_low)-1):
        for j in range(i, len(df_high)):
            change_perc = (df_high[j]-df_low[i])/df_low[i] * 100
            if change_perc > max_change_perc:
                max_change_perc = change_perc
        
    focus_list_3Month[ticker] = change_perc"""
    """if change_perc >= 20 :
        focus_list_3Month[ticker] = change_perc"""
    
final_focus = set(focus_list_ADR.keys()) & set(focus_list_vol.keys()) & set(focus_list_EMA10) & set(focus_candleSize)
print(final_focus)

reading from file:  yahoo_data/US/SPY2023-12-012024-05-12.csv
reading from file:  yahoo_data/US/POWL2023-12-012024-05-12.csv
set()


In [9]:
(1+ADR/300)*TA.EMA(df, period = 10).tail(1).iloc[-1]

161.97005999629056

In [6]:
set(focus_list_ADR.keys()) & set(focus_candleSize)  & set(focus_list_EMA10)

set()

In [7]:
focus_list_EMA10

[]